In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/1029_lstm_최종/data_pd.zip"
extract_folder = "/content/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [ ]:

with open("/content/X_data_pd.pkl", "rb") as f:
    X = pickle.load(f)
with open("/content/y_data_pd.pkl", "rb") as f:
    y = pickle.load(f)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)


(4617388, 10, 23) (4617388,)


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization

# 모델 정의
lstm_model = Sequential()

# 첫 번째 LSTM 레이어
lstm_model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
lstm_model.add(Dropout(0.2))  # 드롭아웃 추가
lstm_model.add(BatchNormalization())  # 배치 정규화 추가

# 두 번째 LSTM 레이어
lstm_model.add(LSTM(150, return_sequences=True))
lstm_model.add(Dropout(0.2))  # 드롭아웃 추가
lstm_model.add(BatchNormalization())  # 배치 정규화 추가

# 세 번째 LSTM 레이어
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.2))  # 드롭아웃 추가
lstm_model.add(BatchNormalization())  # 배치 정규화 추가

# 출력 레이어
lstm_model.add(Dense(1, activation='linear'))

# 모델 컴파일
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 요약
lstm_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 200)           179200    
                                                                 
 dropout (Dropout)           (None, 10, 200)           0         
                                                                 
 batch_normalization (Batch  (None, 10, 200)           800       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 10, 150)           210600    
                                                                 
 dropout_1 (Dropout)         (None, 10, 150)           0         
                                                                 
 batch_normalization_1 (Bat  (None, 10, 150)           600       
 chNormalization)                                       

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint 콜백 생성
checkpoint_filepath = '/content/drive/MyDrive/1029_lstm_최종/lstm_pd.h5'  # 모델을 저장할 파일 경로
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


# EarlyStopping 콜백 생성
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,  # 5 에포크 동안 개선이 없으면 훈련 종료
    restore_best_weights=True)  # 가장 좋은 가중치로 복원



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# 모델 훈련
history = lstm_model.fit(X_train, y_train, epochs=30, batch_size=32,
                        validation_data=(X_test, y_test),
                        callbacks=[model_checkpoint_callback, early_stopping_callback])


# 가장 좋은 결과를 보인 모델 로드
lstm_model.load_weights(checkpoint_filepath)

# 테스트 데이터에 대한 예측 수행
y_pred = lstm_model.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {np.sqrt(mse)}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2 ): {r2}')

y_train_pred = lstm_model.predict(X_train)

# train 평가
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

print(f'Mean Squared Error_train (MSE_train): {mse_train}')
print(f'Root Mean Squared Error_train (RMSE_train): {np.sqrt(mse_train)}')
print(f'Mean Absolute Error_train (MAE_train): {mae_train}')
print(f'R-squared_train (R2_train ): {r2_train}')


# 그래프 그리기
plt.figure(figsize=(10, 6))
plt.scatter(y_pred,y_test)
plt.title('Line Graph of y_pred')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()

lstm_pred = lstm_model.predict(X)

# 전체 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(lstm_pred)
plt.plot(y)
plt.title('Line Graph of total pred')
plt.show()
